In [1]:
!pip install xgboost
!pip install imbalanced-learn
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn import preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 18.6 MB/s eta 0:00:00


In [2]:
train=pd.read_csv("pre_processed_train.csv")
test=pd.read_csv("pre_processed_test.csv")

In [3]:
ytrain=train["isFraud"]
train.drop(axis = 1, labels = ["isFraud","Unnamed: 0"], inplace = True)

In [4]:
test.drop(axis = 1, labels = ["Unnamed: 0"], inplace = True)

In [5]:
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over),('u',under)]
pipeline = Pipeline(steps=steps)
train1, ytrain1 = pipeline.fit_resample(train, ytrain)

In [6]:
from sklearn.preprocessing import StandardScaler
scale= StandardScaler()
X_train = scale.fit_transform(train1)
X_test=scale.transform(test) 

In [7]:
abc = XGBClassifier(n_estimators=5000,
        subsample=0.8,
        max_depth=6,
        objective=["binary:logistic"],
        learning_rate=0.05,
        tree_method='gpu_hist')
abc.fit(X_train,ytrain1)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [9]:
y_pred1 = gs.predict(X_test)
y_pred1.sum()

6046

In [10]:
import csv
with open('xgboost.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id","isFraud"])
    idx=0
    for i in y_pred1:
        writer.writerow([idx,i])
        idx=idx+1